# CS202: Compiler Construction

## In-class Exercises, Week of 01/24/2022

----
# PART I: Lmin language & interpreter; x86 ASTs

## Question 1

Write code to parse the following `Lmin` program and print out its abstract syntax tree.

In [2]:
from ast import *
from cs202_support.base_ast import print_ast

program = "print(42)"

In [4]:
ast = Module(
 [
  Expr(
   Call(
    Name(
     "print",
     Load()),
    [
     Constant(
      42,
      None)
    ],
    []))
 ],
 [])
print(print_ast(ast))

Module(
 [
  Expr(
   Call(
    Name(
     "print",
     Load()),
    [
     Constant(
      42,
      None)
    ],
    []))
 ],
 [])


## Question 2

Write an interpreter `eval_lmin` for the `Lmin` language.

In [5]:
def eval_lmin(program):
    match program:
        case Module([Expr(Call(Name("print",Load()),[Constant(n,None)],[]))],[]):
            print(n)


eval_lmin(ast)

42


## Question 3

Write code to generate a *pseudo-x86 abstract syntax tree* for the `start` block for the program above.

Hint: reference the [pseudo-x86 AST class hierarchy](https://github.com/jnear/cs202-assignments/blob/master/cs202_support/x86exp.py). Debug your solution using the online compiler's output for the `select instructions` pass.

In [6]:
import cs202_support.x86exp as x86

ast = x86.Program(
 {
  'start':
   [
    x86.NamedInstr(
     "movq",
     [
      x86.Immediate(42),
      x86.Reg("rdi")
     ]),
    x86.Callq("print_int"),
    x86.Jmp("conclusion")
   ]
 })


print(print_ast(ast))

Program(
 {
  'start':
   [
    NamedInstr(
     "movq",
     [
      Immediate(42),
      Reg("rdi")
     ]),
    Callq("print_int"),
    Jmp("conclusion")
   ]
 })


# Part II: Passes of the Compiler

## Question 4

What is the purpose of the `select_instructions` pass of the compiler? How should it be implemented?

It selects which x86 (or other) instructions should be used based on the program. It turns our language into a language that has a 1-to-1 translation to machine code, which is our ultimate destination. It should be implemented to take in a language ast and return an assembly ast. It should generally be implemented recursively, though in this trivial case it suffices to include only one check.

## Question 5

What is the purpose of the `print_x86` pass of the compiler? How should it be implemented?

To turn an assembly ast into an assembly string which can be assembled into machine code. Its purpose is to take an ast, which is a programmer-friendly construct, into a string, a backwards-compatible construct. It should be implemented recursively.

# Part III: Lvar

## Question 6

Write an interpreter `eval_lvar` for the `Lvar` language. Reference the grammar: Figure 2.2 (page 14) in the textbook.

In [30]:
from typing import Dict

def eval_lvar(program: Module):

    def input_int():
        return int(input())

    env = {
        'print': print,
        'input_int': input_int
    }

    def interp_exp(e, env):
        match e:
            case Name(var, Load()):
                return env[var]
            case Constant(i):
                return i
            case UnaryOp(USub(), i):
                return - interp_exp(i, env)
            case BinOp(i1, op, i2):
                match op:
                    case Add():
                        return interp_exp(i1, env) + interp_exp(i2, env)
                    case Sub():
                        return interp_exp(i1, env) - interp_exp(i2, env)
            case Call(name, args):
                f = interp_exp(name, env)
                f(*[interp_exp(arg, env) for arg in args])
                return None
            case _:
                raise Exception("interp_exp")

    def interp_stmt(s, env):
        match s:
            case Expr(exp):
                interp_exp(exp, env)
            case Assign([Name(var, Store())], i):
                env[var] = interp_exp(i, env)
            case _:
                raise Exception("interp_exp")

    def interp_module(m, env):
        match m:
            case Module(statements):
                for statement in statements:
                    interp_stmt(statement, env)

    interp_module(program, env)





In [31]:
# TEST CASE
program = """
x = 5
y = 6
print(x + y)"""

eval_lvar(parse(program))

11


----
# PART IV: Remove Complex Operands

/## Question 7

Consider this translation of an expression to assembly language. What is wrong with it?

In [44]:
python = """
x = 1 + 2 + 3
"""

asm = """
movq $2, %rax
addq $1, (addq $3, %rax)
"""

it has a nested addq, which is not allowed in assembly.

## Question 8

Which AST nodes in the language `Lvar` are **atomic**?

integer constants and variables

## Question 9

Why do we need this pass? What is the form of its output?

<<YOUR ANSWER>>

## Question 10

Convert the program from earlier into A-normal form.

In [45]:
python = """
x = 1 + 2 + 3
"""

In [46]:
python_anf = """
<<YOUR ANSWER>>
"""

## Question 11

Describe a recursive procedure to perform the *remove-complex-opera* pass. Reference section 2.4 in the textbook.

<<YOUR ANSWER>>